<a href="https://colab.research.google.com/github/relfarizi/Final_Project/blob/master/FinalDrugDiscoveryUpdateLagiRevisi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Import Library

### - Install RDKIT

In [ ]:
# Install rdkit

def install_rdkit():
  !wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
  !chmod +x Miniconda3-latest-Linux-x86_64.sh
  !time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
  !time conda install -q -y -c conda-forge rdkit

install_rdkit()

### - Library

In [ ]:
# Data

from google.colab import files
import pandas as pd
import numpy as np
import regex as re

# rdkit
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
from rdkit import Chem
from rdkit.Chem import DataStructs,AllChem

# random
import random as rd

import matplotlib.pyplot as plt

import math

## 2. Data

### Pre-Processing Data

In [ ]:
def preprocessing_data(link_of_data) :
  dataset = pd.read_csv(link_of_data,sep=";")
  dataset = dataset.iloc[:,[0,7,9,10,11]]

  sameMol = []
  sameMolCount = {}
  dataset_handle = []

  # drop Nan value
  mol_smiles = dataset.iloc[:,1].values.tolist()
  row_drop = []
  for i in range(len(mol_smiles)) :
    if len(str(mol_smiles[i])) < 5 :
      row_drop.append(i)
  #dfNan = dataset.iloc[row_drop,:]
  print(row_drop)
  print(len(dataset))  
  #dataset = dataset.drop(row_drop)
  print(len(dataset))
  molDup = []
  molDup1 = []
  rowDrop = []

  for idx,x in enumerate(dataset.values):
    mol_name = x[0]
    if mol_name in sameMol:
      #print(mol_name)
      if idx not in row_drop :
        row_drop.append(idx)
    else:
      sameMol.append(mol_name)
  print("sameMol ",len(dataset) - len(sameMol))
  print("rowdrop ",len(row_drop))
  print(dataset.iloc[162,:])
  dataset = dataset.drop(row_drop)
  print(dataset)

  #print(dataset_handle)
    
  list_of_column_dataframe = dataset.columns
  list_of_dataset_after = []

  list_of_dataset_before = np.array(dataset)
  
  for data in list_of_dataset_before :
    if data[2] == "'='" :
      if data[4] == "nM" :
        if data[3] < 10000 :
          data[3] = data[3]/1000
          list_of_dataset_after.append(data)

  dataframe_of_dataset_after = pd.DataFrame(list_of_dataset_after)
  dataframe_of_dataset_after.columns = list_of_column_dataframe
  name_of_file_download = "Dataset_After1.csv"
  dataframe_of_dataset_after.to_csv(name_of_file_download)
  #files.download(name_of_file_download)
  return dataframe_of_dataset_after

# Run untuk mendapatkan datase setelah preprocessing
#preprocessing_data("https://raw.githubusercontent.com/relfarizi/Final_Project/master/Data/Dataset_Before.csv")
#preprocessing_data("Data/Dataset_Before.csv")

### Data After Pre-Processing

In [ ]:
# for colab

dataset = pd.read_csv('https://raw.githubusercontent.com/relfarizi/Final_Project/master/Dataset_After1.csv', index_col= 0)


#dataset = pd.read_csv('Dataset_After1.csv',index_col = 0)
print(len(dataset))
dataset.head(5)

In [ ]:
ml =dataset["Molecule ChEMBL ID"].tolist()
molTmp = []
sum = 0
for mol in ml :
    if mol in molTmp :
        sum += 1
    else :
        molTmp.append(mol)
print(sum)

## 3. Fitur yang digunakan

variabel yang digunakan :
* mol_name = list nama molekul
* mol_smile = list smile molekul
* mol_act = list berisi nilai IC50
* mol_fp = list yang berisi fingerprint dari setiap molekul

In [ ]:
def dataframe(dataset,opt=0):

  if opt == 0 :
    mol_name = dataset.iloc[:,0].values.tolist()
    mol_smiles = dataset.iloc[:,1].values.tolist()
    mol_act = dataset.iloc[:,3].values.tolist()
    
  elif opt == 1:
    mol_name = dataset["mol_name"].tolist()
    mol_smiles = dataset["mol_smiles"].tolist()
    mol_act = dataset["mol_act"].tolist()

  mol_fp = []
  for molekul in mol_smiles :
    smiles = Chem.MolFromSmiles(str(molekul))
    fp = Chem.RDKFingerprint(smiles)
    #fp =  AllChem.GetMorganFingerprintAsBitVect(smiles,2,nBits=1024)
    mol_fp.append(fp)

  sum_act = 0
  for molekul in mol_act :
    act = np.exp(-molekul)
    #print(act)
    sum_act  += act
    
  n = len(mol_act)
  #p_avg = 1/n
  mol_prob = []

  # act = probabilitas kesuksesan
  #print(sum_act)
  sum_pro = 0
  for molekul in mol_act :
    #act = (molekul[1]*p_avg*n)/sum_act
    act = np.exp(-molekul)
    prob = act/sum_act
    prob = round(prob,8)
    prob = act
    mol_prob.append(prob)
    sum_pro += prob
  print(sum_pro)

  mol_ex = []
  G = 100000
  c = 250
  for molekul in mol_prob :
    ex = ((G/c)*molekul) - 1
    mol_ex.append(ex)

  df = pd.DataFrame({'mol_name':mol_name,
                       'mol_smiles' : mol_smiles,
                       'mol_act':mol_act,
                       'mol_fp' : mol_fp,
                       'mol_prob' :mol_prob,
                       'mol_ex' : mol_ex

                       })
  return df


In [ ]:
df = dataframe(dataset)

In [ ]:
df

In [ ]:
df_mol = df.iloc[:,[0,2,3,4,5]]
lsAct = df_mol.iloc[:,1].tolist()
#lsAct = sorted(lsAct)
#x = [i for i in range(len(lsAct))]
x = df_mol.iloc[:,4].tolist()

plt.xlabel('IC50', fontsize=15)
plt.ylabel('Probability of Success', fontsize=15)
plt.scatter(lsAct, x)
plt.show()

## 4. Algoritma NSGA II

### Supurt function

#### Display

#### Cek Duplikat Individu
Melakukan pengecekan terhadap individu yang sama (mempunyai keseluruhan anggota yang sama) dalam suatu populasi

---
Contoh :


```
a = [1,2,3,4,5]
b = [2,3,4,5,7]
c = [1,5,2,3,4]

pop = [a,b,c]

```
Maka :
```
isDupInd(c,pop) ==> True
isDupInd(b,pop) ==> False
```
---





In [ ]:
def isDupInd(individu,population,dfMol = df_mol) :
    same = False
    molName = dfMol["mol_name"]
    molNameList = []
    for gen in individu :
        molNameList.append(gen)
    individu = molNameList
    sizeList = len(individu)
    #print(sizeList)
    count = 0
    for ind in population :
        ind = ind[0]
        for gen in individu :
            if gen in ind :
                count += 1
        
        if count == sizeList :
            #print(count)
            #print("masuk")
            same = True
            break
        else :
            count = 0
    return same


#### Sort by Value
Me-return list berupa index dari element yang sudah terurut(descend)

In [ ]:
def index_of(a,list):
    for i in range(0,len(list)):
        if list[i] == a:
            return i
    return -1

In [ ]:
import math
def sort_by_values(list1, values):
    sorted_list = []
    while(len(sorted_list)!=len(list1)):
        if index_of(min(values),values) in list1:
            sorted_list.append(index_of(min(values),values))
        values[index_of(min(values),values)] = math.inf
    return sorted_list


In [ ]:
def sortByValue(ls) :
    sort = sorted((value,index) for index,value in enumerate(ls))
    sortLs = []
    for x in sort :
        sortLs.append((x[1]))
    return sortLs
#sortByValue(ls)

#### bundling Individu
Menghitung nilai activity dan diversity dari masing - masing Individu

In [ ]:
def bundleIndividu(individu,dfMol = df_mol):
    molName = dfMol["mol_name"].tolist()
    molAct = dfMol["mol_act"].tolist()
    molFp = dfMol["mol_fp"].tolist()
    molProb = dfMol["mol_prob"].tolist()
 
    fp_list = []
    n = len(molName)

    def fObjective(individu = individu,dfMol = df_mol) :
    
        def fObj1(individu = individu,dfMol = dfMol) :
            act = []
            molName = dfMol["mol_name"].tolist()

            cAvg = 3
            B = len(individu) * cAvg
            G = len(individu) * 1000
            for idx,x in enumerate(molName) :
                if x  in individu :
                    act.append(dfMol["mol_prob"][idx]*G)
            return np.sum(act) - B
        
        def fObj2(individu = individu, dfMol = dfMol) :
            fpList = []
            molFp = []
            molFpName = []
            molName = dfMol["mol_name"].tolist()
            molFpLs = dfMol["mol_fp"].tolist()
            for idx,x in enumerate(molName) :
                if x in individu :
                    molFpName.append(x)
                    molFp.append(molFpLs[idx])
            n = len(molFp)
            #print("n=",n,len(individu))
            sumDiverse = 0
            divLs = np.zeros([n,n])
            
            for i in range(n):
                 # Update code
                for j in range(n):
                    simT = DataStructs.FingerprintSimilarity(molFp[i],molFp[j])
                    dx = 1 - simT
                    tetha = 0.5
                    fx = np.exp((-1)*tetha*dx)
                    #fx = round(fx,2)
                    divLs[i][j] = fx
            
            #handle determinan = 0
            handleDet = 0.0000001
            countDet = 1
            sameCorr = []
            for i in range(n) :
                for j in range(n) :
                    if i > j :

                        if divLs[i][j] in sameCorr :
                            #print(divLs[i][j])
                            a = divLs[i][j] - (divLs[i][j]*countDet*handleDet)
                            #print(divLs[i][j],a)
                            divLs[i][j] = a
                            divLs[j][i] = a
                            countDet += 1
                        else :
                            sameCorr.append(divLs[i][j])



            #print(divLs)
            #print(np.linalg.det(divLs))
            #print(divLs)
            e = np.ones(n)
            eT = np.transpose(e)
            divLsInv = np.linalg.inv(divLs)
            sumDiverse = e.dot(divLsInv).dot(eT)
            #print(divLsInv)
            #sumDiverse = np.sum(divLsInv)

            # update code
            sumdiverse = 0
            """for i in divLsInv :
                for j in i :
                    sumDiverse += abs(j)
                    #print(abs(j))"""
            
            #print(sumDiverse)
            #sumDiverse = 50
            return sumDiverse


        return fObj1(), fObj2()
    fObj1,fObj2 = fObjective()
    
    if [individu,fObj1,fObj2] is not None :
        return [individu,fObj1,fObj2]
    else :
        return []


#### Extract Individu
mengekstract nilai pada setiap individu yang terdapat pada 1 populasi

In [ ]:
def unpackInd(population) :
    fMaxObj1 = []
    fMaxObj2 = []
    solution = []
    for idx,individu in enumerate(population) :
        #print(idx)
        solution.append(individu[0])
        fMaxObj1.append(individu[1]) # Probabilitas Kesuksesan
        fMaxObj2.append(individu[2]) # Diversity   
    return [solution,fMaxObj1,fMaxObj2]

### Inisialisasi populasi

Pada proses ini akan dihasilkan sekumpulan nPortfolio individu (Portfolio). Setiap individu terdiri dari nGen Gen (Molekul)

In [ ]:
def initPopulation(nPortfolio,nGen,dfMol=df_mol):
    def initIndividu(dfMol = df_mol,nGen=nGen):
        randIndividu = []
        while(len(randIndividu) < nGen) :
            gen = rd.randint(0,len(dfMol)-1)
            if gen not in randIndividu:
                randIndividu.append(gen)
        #print(len(randIndividu))
        return randIndividu

    molName = dfMol["mol_name"].tolist()
    molAct = dfMol["mol_act"].tolist()
    molFp = dfMol["mol_fp"].tolist()
    molProb = dfMol["mol_prob"].tolist()
    population = []
    
    nSubset = dfMol.shape[0]

    for i in range(nPortfolio) :
        randIndividu = initIndividu()

        if i > 0 :
            while(isDupInd(randIndividu,population)) :
                randIndividu = initIndividu()

        molNameList = []
        for j in randIndividu :
            molNameList.append(molName[j])     
        individu = molNameList
        bInd = bundleIndividu(individu)
        population.append(bInd)
    return population

### Generate Offspring ( crossover and mutation )
Pada proses ini akan dihasilkan sekumpulan individu sebanyak 2 kali lipat dari populasi sebelumnya. Untuk mendapatkannya dilakukan :

*   Crossover
*   Mutation

Hal yang harus dihindari : Pengulangan individu yang sama dalam 1 populasi baru



In [ ]:
def generateOffspring(population,dfMol = df_mol,pCo = 0.8,pMut=0.05) :
    sizeOfPop = len(population)

    #cek gen yang sama untuk menghindari duplikasi pd crossover
    def duplicateMol(parent) :
        parent1,parent2 = parent
        dupGen = []
        for gen in parent1 :
            if gen in parent2 :
                dupGen.append(gen)
        return dupGen

    # return 1 pasangan parent
    def generateParent(population=population, sizeOfPop = sizeOfPop):
        parent1 = population[rd.randint(0,sizeOfPop-1)][0]
        parent2 = population[rd.randint(0,sizeOfPop-1)][0]
        return [parent1,parent2]

    def crossover(parent,pCo = pCo) :
       
        randCo = rd.random()
        parents = generateParent()
        parent1,parent2 = (parents)
        dupMol = ((duplicateMol(parents)))

        # pemotongan parent dengan gen yang sama
        
        parent1 = list(set(parent1)-set(dupMol))
        parent2 = list(set(parent2)-set(dupMol))
        nGen = len(parent1)
        
        nPoint = 0.5
        barier = math.floor(nGen*nPoint) 

        if randCo < pCo :
            offspring1 = parent1[:barier]+parent2[barier:]
            offspring2 = parent2[:barier]+parent1[barier:]
        else :
            offspring1 = parent1
            offspring2 = parent2
        offspring1 += dupMol
        offspring2 += dupMol
        return ([offspring1,offspring2])

   # Flip mutation 
    def mutation(individu,pMut = pMut,pMr = 0.5,dfMol = dfMol) :
        randMut = rd.uniform(0,1)
        newOffspring = []
        if randMut < pMut :
            newInd = []
            molName = [mol for mol in dfMol["mol_name"] if mol not in individu]
            for idx,gen in enumerate(individu) :
                randRep = rd.uniform(0,1)
                if randRep > pMr :
                    newMol = rd.choice(molName)
                    while(newMol in individu) :
                        individu[idx] = rd.choice(molName)


        return individu



    
    # Start Generate
    solution = []
    while(len(solution) < 2*sizeOfPop):
        parents = generateParent()
        offSpring = crossover(parents)
        for x in offSpring :
            if len(solution) > 0 :
                if not isDupInd(x,solution) :
                    solution.append(bundleIndividu(mutation(x),dfMol))
                else :
                    continue
            else :
                 solution.append(bundleIndividu(mutation(x),dfMol))
    return solution


### Non-Dominated Sorted

Proses Sorting berdasarkan prinsip non dominance antar fungsi

In [ ]:
def fast_non_dominated_sort(values1, values2):
    S=[[] for i in range(0,len(values1))]
    front = [[]]
    n=[0 for i in range(0,len(values1))]
    rank = [0 for i in range(0, len(values1))]

    for p in range(0,len(values1)):
        S[p]=[]
        n[p]=0
        for q in range(0, len(values1)):
            if (values1[p] > values1[q] and values2[p] > values2[q]) or (values1[p] >= values1[q] and values2[p] > values2[q]) or (values1[p] > values1[q] and values2[p] >= values2[q]):
                if q not in S[p]:
                    S[p].append(q)
            elif (values1[q] > values1[p] and values2[q] > values2[p]) or (values1[q] >= values1[p] and values2[q] > values2[p]) or (values1[q] > values1[p] and values2[q] >= values2[p]):
                n[p] = n[p] + 1
        if n[p]==0:
            rank[p] = 0
            if p not in front[0]:
                front[0].append(p)

    i = 0
    while(front[i] != []):
        Q=[]
        for p in front[i]:
            for q in S[p]:
                n[q] =n[q] - 1
                if( n[q]==0):
                    rank[q]=i+1
                    if q not in Q:
                        Q.append(q)
        i = i+1
        front.append(Q)

    del front[len(front)-1]
    return front



### Crowding Distance
Untuk meng-eliminasi beberapa individu yang mempunyai perfomansi sama

In [ ]:
def crowding_distance(values1, values2, front):
    distance = [0 for i in range(0,len(front))]
    sorted1 = sort_by_values(front, values1[:])

    sorted2 = sort_by_values(front, values2[:])
    
    distance[0] = math.inf
    distance[len(front) - 1] = math.inf
    for k in range(1,len(front)-1):

        distance[k] = distance[k]+ (values1[sorted1[k+1]] - values2[sorted1[k-1]])/(max(values1)-min(values1))
        
    for k in range(1,len(front)-1):
        distance[k] = distance[k]+ (values1[sorted2[k+1]] - values2[sorted2[k-1]])/(max(values2)-min(values2))
    return distance

### Main Program

In [ ]:
# Parameter
import matplotlib.pyplot as plt

from datetime import datetime
startTime = datetime.now()

maxGen = 100
gen_0 = 0
nGen = 10
nPortfolio = 100 # banyaknya individu dalam satu populasi


# Step 1 Generate Population
population = initPopulation(nPortfolio,nGen)
initFobj1 = unpackInd(population)[1]
initFobj2 = unpackInd(population)[2]

print("Start")
for gen in range(maxGen) :
    #print("gen ", gen+1, "Start!")

    # Step 2 Generate Offsprings
   
    offsprings = generateOffspring(population)
    fObj1 = unpackInd(offsprings)[1]
    fObj2 = unpackInd(offsprings)[2]
    #print(len(fObj1))
    
    # Step 3 Fast Non Dominated
    nonDominatedSorted = fast_non_dominated_sort(fObj1[:],fObj2[:])
    
    # Step 4 crowding distance
    crowdingDistance = []
    for i in range(len(nonDominatedSorted)) :
        crowdingDistance.append(crowding_distance(fObj1[:],fObj2[:],nonDominatedSorted[i][:]))
    

    # Step 5 Individu selection
    newNonDominatedSorted = []
    new_solution = []

    for i in range(0,len(nonDominatedSorted)):  

        # step 5.1 assign sort by index dari crowding distance

        front22 = sortByValue(crowdingDistance[i][:])
        if gen == 1 :
            cSave = crowdingDistance[i][:]
            fSave = front22

        # step 5.2 memasukkan nilai dari non_dominated .. berdasrkan urutan crowding

        front = []
        for j in range(len(nonDominatedSorted[i])):
            front.append(nonDominatedSorted[i][front22[j]])

        # Step 5.3 masukin ke new_sol sesuai dengan keperluan pop_size

        for value in front:
            new_solution.append(value)
            
            if(len(new_solution)==nPortfolio):
                break 
                
        if (len(new_solution) == nPortfolio):
            break
    
    population = [offsprings[i] for i in new_solution]

    maxFObj1 = unpackInd(population)[1]
    #print(maxFObj1)
    maxFObj2 = unpackInd(population)[2]

    plt.title("gen "+str(gen+1))
    plt.xlabel('Diversity', fontsize=15)
    plt.ylabel('Activtity', fontsize=15)
    plt.scatter(maxFObj2, maxFObj1)
    
    plt.show()
    #clear_output(wait=True)
    

 
print("Running Time = ",datetime.now() - startTime)
    

In [ ]:
sumGen = [1000]
sizeGen = 1000
from datetime import datetime
startTime = datetime.now()
print("START maxgen",sizeGen)

maxGen = sizeGen
gen_0 = 0
nGen = 5
nPortfolio = 100

iteration = 0
print("Start percobaan ke-",iteration+1)
# Step 1 Generate Population
population = initPopulation(nPortfolio,nGen)
initFobj1 = unpackInd(population)[1]
initFobj2 = unpackInd(population)[2]
for gen in range(maxGen) :
    print("gen ", gen+1, "Start!")

    # Step 2 Generate Offsprings

    offsprings = generateOffspring(population)
    fObj1 = unpackInd(offsprings)[1]
    fObj2 = unpackInd(offsprings)[2]
    #print(len(fObj1))
    
    # Step 3 Fast Non Dominated
    nonDominatedSorted = fast_non_dominated_sort(fObj1[:],fObj2[:])
    
    # Step 4 crowding distance
    crowdingDistance = []
    for i in range(len(nonDominatedSorted)) :
        crowdingDistance.append(crowding_distance(fObj1[:],fObj2[:],nonDominatedSorted[i][:]))
    

    # Step 5 Individu selection
    newNonDominatedSorted = []
    new_solution = []

    for i in range(0,len(nonDominatedSorted)):  

        # step 5.1 assign sort by index dari crowding distance

        front22 = sortByValue(crowdingDistance[i][:])
        if gen == 1 :
            cSave = crowdingDistance[i][:]
            fSave = front22

        # step 5.2 memasukkan nilai dari non_dominated .. berdasrkan urutan crowding

        front = []
        for j in range(len(nonDominatedSorted[i])):
            front.append(nonDominatedSorted[i][front22[j]])

        # Step 5.3 masukin ke new_sol sesuai dengan keperluan pop_size

        for value in front:
            new_solution.append(value)
            
            if(len(new_solution)==nPortfolio):
                break
                
        if (len(new_solution) == nPortfolio):
            break
    
    population = [offsprings[i] for i in new_solution]

    maxFObj1 = unpackInd(population)[1]
    maxFObj2 = unpackInd(population)[2]

graf = {"x":maxFObj1,"y":maxFObj2}
dfGraf = pd.DataFrame(graf)
dfGraf.to_csv("DataRevisi"+str(maxGen)+"iter/Testing6_"+str(iteration+1)+"_"+str(nGen)+".csv")
files.download("DataRevisi"+str(maxGen)+"iter/Testing6_"+str(iteration+1)+"_"+str(nGen)+".csv")

print("Testing",iteration + 1,sizeGen,"Saved !")
        #clear_output(wait=True)
print("Running Time = ",datetime.now() - startTime)

In [ ]:
dfGraf.to_csv("DataRevisi"+str(maxGen)+"iter/Testing6_"+str(iteration+1)+"_"+str(nGen)+".csv")
files.download("DataRevisi"+str(maxGen)+"iter/Testing6_"+str(iteration+1)+"_"+str(nGen)+".csv")

In [ ]:
# Parameter
import matplotlib.pyplot as plt
maxGen = 50
gen_0 = 0
nGen = 5
nPortfolio = 100 # banyaknya individu dalam satu populasi


# Step 1 Generate Population
population = initPopulation(nPortfolio,nGen)
initFobj1 = unpackInd(population)[1]
initFobj2 = unpackInd(population)[2]

print("Start")
for gen in range(maxGen) :
    #print("gen ", gen+1, "Start!")

    # Step 2 Generate Offsprings
   
    offsprings = generateOffspring(population)
    fObj1 = unpackInd(offsprings)[1]
    fObj2 = unpackInd(offsprings)[2]
    print(len(fObj1))
    
    # Step 3 Fast Non Dominated
    nonDominatedSorted = fast_non_dominated_sort(fObj1[:],fObj2[:])
    
    # Step 4 crowding distance
    crowdingDistance = []
    for i in range(len(nonDominatedSorted)) :
        crowdingDistance.append(crowding_distance(fObj1[:],fObj2[:],nonDominatedSorted[i][:]))
    

    # Step 5 Individu selection
    newNonDominatedSorted = []
    new_solution = []

    for i in range(0,len(nonDominatedSorted)):  

        # step 5.1 assign sort by index dari crowding distance

        front22 = sortByValue(crowdingDistance[i][:])
        if gen == 1 :
            cSave = crowdingDistance[i][:]
            fSave = front22

        # step 5.2 memasukkan nilai dari non_dominated .. berdasrkan urutan crowding

        front = []
        for j in range(len(nonDominatedSorted[i])):
            front.append(nonDominatedSorted[i][front22[j]])

        # Step 5.3 masukin ke new_sol sesuai dengan keperluan pop_size

        for value in front:
            new_solution.append(value)
            
            if(len(new_solution)==nPortfolio):
                break
                
        if (len(new_solution) == nPortfolio):
            break
    
    population = [offsprings[i] for i in new_solution]

    maxFObj1 = unpackInd(population)[1]
    maxFObj2 = unpackInd(population)[2]

    plt.title("gen "+str(gen+1))
    plt.xlabel('Diversity', fontsize=15)
    plt.ylabel('Activtity', fontsize=15)
    plt.scatter(maxFObj2, maxFObj1)
    
    plt.show()
    #clear_output(wait=True)

In [ ]:
for iteration in range(30) :
    maxGen = 50
    gen_0 = 0
    nGen = 25
    nPortfolio = 100
    print("Start percobaan ke-",iteration+1)
    # Step 1 Generate Population
    population = initPopulation(nPortfolio,nGen)
    initFobj1 = unpackInd(population)[1]
    initFobj2 = unpackInd(population)[2]
    for gen in range(maxGen) :
        print("gen ", gen+1, "Start!")

        # Step 2 Generate Offsprings
    
        offsprings = generateOffspring(population)
        fObj1 = unpackInd(offsprings)[1]
        fObj2 = unpackInd(offsprings)[2]
        #print(len(fObj1))
        
        # Step 3 Fast Non Dominated
        nonDominatedSorted = fast_non_dominated_sort(fObj1[:],fObj2[:])
        
        # Step 4 crowding distance
        crowdingDistance = []
        for i in range(len(nonDominatedSorted)) :
            crowdingDistance.append(crowding_distance(fObj1[:],fObj2[:],nonDominatedSorted[i][:]))
        

        # Step 5 Individu selection
        newNonDominatedSorted = []
        new_solution = []

        for i in range(0,len(nonDominatedSorted)):  

            # step 5.1 assign sort by index dari crowding distance

            front22 = sortByValue(crowdingDistance[i][:])
            if gen == 1 :
                cSave = crowdingDistance[i][:]
                fSave = front22

            # step 5.2 memasukkan nilai dari non_dominated .. berdasrkan urutan crowding

            front = []
            for j in range(len(nonDominatedSorted[i])):
                front.append(nonDominatedSorted[i][front22[j]])

            # Step 5.3 masukin ke new_sol sesuai dengan keperluan pop_size

            for value in front:
                new_solution.append(value)
                
                if(len(new_solution)==nPortfolio):
                    break
                    
            if (len(new_solution) == nPortfolio):
                break
        
        population = [offsprings[i] for i in new_solution]

        maxFObj1 = unpackInd(population)[1]
        maxFObj2 = unpackInd(population)[2]

    graf = {"x":maxFObj1,"y":maxFObj2}
    dfGraf = pd.DataFrame(graf)
    dfGraf.to_csv("DataGrafik1/Testing_"+str(iteration+1)+"_"+str(nGen)+".csv")
for iteration in range(30) :
    maxGen = 50
    gen_0 = 0
    nGen = 10
    nPortfolio = 100
    print("Start percobaan ke-",iteration+1)
    # Step 1 Generate Population
    population = initPopulation(nPortfolio,nGen)
    initFobj1 = unpackInd(population)[1]
for iteration in range(30) :
    maxGen = 50
    gen_0 = 0
    nGen = 10
    nPortfolio = 100
    print("Start percobaan ke-",iteration+1)
    # Step 1 Generate Population
    population = initPopulation(nPortfolio,nGen)
    initFobj1 = unpackInd(population)[1]
    initFobj2 = unpackInd(population)[2]
    #clear_output(wait=True)

In [ ]:
# Parameter
import matplotlib.pyplot as plt
maxGen = 50
gen_0 = 0
nGen = 25
nPortfolio = 100 # banyaknya individu dalam satu populasi




for iteration in range(30) :
    maxGen = 50
    gen_0 = 0
    nGen = 100
    nPortfolio = 100
    print("Start percobaan ke-",iteration+1)
    # Step 1 Generate Population
    population = initPopulation(nPortfolio,nGen)
    initFobj1 = unpackInd(population)[1]
    initFobj2 = unpackInd(population)[2]
    for gen in range(maxGen) :
        print("gen ", gen+1, "Start!")

        # Step 2 Generate Offsprings
    
        offsprings = generateOffspring(population)
        fObj1 = unpackInd(offsprings)[1]
        fObj2 = unpackInd(offsprings)[2]
        #print(len(fObj1))
        
        # Step 3 Fast Non Dominated
        nonDominatedSorted = fast_non_dominated_sort(fObj1[:],fObj2[:])
        
        # Step 4 crowding distance
        crowdingDistance = []
        for i in range(len(nonDominatedSorted)) :
            crowdingDistance.append(crowding_distance(fObj1[:],fObj2[:],nonDominatedSorted[i][:]))
        

        # Step 5 Individu selection
        newNonDominatedSorted = []
        new_solution = []

        for i in range(0,len(nonDominatedSorted)):  

            # step 5.1 assign sort by index dari crowding distance

            front22 = sortByValue(crowdingDistance[i][:])
            if gen == 1 :
                cSave = crowdingDistance[i][:]
                fSave = front22

            # step 5.2 memasukkan nilai dari non_dominated .. berdasrkan urutan crowding

            front = []
            for j in range(len(nonDominatedSorted[i])):
                front.append(nonDominatedSorted[i][front22[j]])

            # Step 5.3 masukin ke new_sol sesuai dengan keperluan pop_size

            for value in front:
                new_solution.append(value)
                
                if(len(new_solution)==nPortfolio):
                    break
                    
            if (len(new_solution) == nPortfolio):
                break
        
        population = [offsprings[i] for i in new_solution]

        maxFObj1 = unpackInd(population)[1]
        maxFObj2 = unpackInd(population)[2]

    graf = {"x":maxFObj1,"y":maxFObj2}
    dfGraf = pd.DataFrame(graf)
    dfGraf.to_csv("DataGrafik/Testing_"+str(iteration+1)+"_"+str(nGen)+".csv")
# Parameter
import matplotlib.pyplot as plt
maxGen = 50
gen_0 = 0
nGen = 25
nPortfolio = 100 # banyaknya individu dalam satu populasi




…
        maxFObj1 = unpackInd(population)[1]
        maxFObj2 = unpackInd(population)[2]

    graf = {"x":maxFObj1,"y":maxFObj2}
    dfGraf = pd.DataFrame(graf)
    dfGraf.to_csv("DataGrafik/Testing_"+str(iteration+1)+"_"+str(nGen)+".csv")
        #clear_output(wait=True)

In [ ]:
f1 = [initFobj1,maxFObj1]
f2 = [initFobj2,maxFObj2]
title = ["Generasi ke 1","Generasi ke "+str(maxGen)]
n = len(f1)
fig, ax = plt.subplots(ncols=n,figsize=(18,5))
fig.tight_layout(pad=5.0)
for i in range(n) :
    ax[i].scatter(f2[i],f1[i])
    ax[i].set(xlabel="Diversity",ylabel="Activity")
    ax[i].set_title(title[i],fontsize=20)

plt.show()

In [ ]:
graf = {"x":maxFObj1,"y":maxFObj2}
dfGraf = pd.DataFrame(graf)
dfGraf.to_csv("fungsi_"+str(nGen)+".csv")

In [ ]:
"""
# Parameter
import matplotlib.pyplot as plt
maxGen = 50
gen_0 = 0
nGen = 10
nPortfolio = 100 # banyaknya individu dalam satu populasi

listMaxGen = [50]
listNgen = [5,10,25,50,100]
listPMut = [0.2,0.4,0.5]
listPCo = [0.2,0.4,0.5]
listPortfolio = [50,100]
# Step 1 Generate Population

for maxGen in listMaxGen :
    for nGen in listNgen :
        for pCo in listPCo :
            for pMut in listPMut :
                for nPortfolio in listPortfolio :
                    gen_0 = 0

                    population = initPopulation(nPortfolio,nGen)
                    initFobj1 = unpackInd(population)[1]
                    initFobj2 = unpackInd(population)[2]

                    print("Start_"+str(maxGen)+"_"+str(nGen)+"_"+str(pCo)+"_"+str(pMut)+"_"+str(nPortfolio))
                    for gen in range(maxGen) :
                        print("gen ", gen+1, "Start!")

                        # Step 2 Generate Offsprings
                    
                        offsprings = generateOffspring(population,pMut = pMut, pCo = pCo)
                        fObj1 = unpackInd(offsprings)[1]
                        fObj2 = unpackInd(offsprings)[2]
                        #print(len(fObj1))
                        
                        # Step 3 Fast Non Dominated
                        nonDominatedSorted = fast_non_dominated_sort(fObj1[:],fObj2[:])
                        
                        # Step 4 crowding distance
                        crowdingDistance = []
                        for i in range(len(nonDominatedSorted)) :
                            crowdingDistance.append(crowding_distance(fObj1[:],fObj2[:],nonDominatedSorted[i][:]))
                        

                        # Step 5 Individu selection
                        newNonDominatedSorted = []
                        new_solution = []

                        for i in range(0,len(nonDominatedSorted)):  

                            # step 5.1 assign sort by index dari crowding distance

                            front22 = sortByValue(crowdingDistance[i][:])
                            if gen == 1 :
                                cSave = crowdingDistance[i][:]
                                fSave = front22

                            # step 5.2 memasukkan nilai dari non_dominated .. berdasrkan urutan crowding

                            front = []
                            for j in range(len(nonDominatedSorted[i])):
                                front.append(nonDominatedSorted[i][front22[j]])

                            # Step 5.3 masukin ke new_sol sesuai dengan keperluan pop_size

                            for value in front:
                                new_solution.append(value)
                                
                                if(len(new_solution)==nPortfolio):
                                    break
                                    
                            if (len(new_solution) == nPortfolio):
                                break
                        
                        population = [offsprings[i] for i in new_solution]

                        maxFObj1 = unpackInd(population)[1]
                        maxFObj2 = unpackInd(population)[2]
                    graf = {"x":maxFObj1,"y":maxFObj2}
                    dfGraf = pd.DataFrame(graf)
                    dfGraf.to_csv("fungsi_"+str(nGen)+"_"+str(pCo)+"_"+str(pMut)+"_"+str(nPortfolio)+".csv")
                    print("End_"+str(maxGen)+"_"+str(nGen)+"_"+str(pCo)+"_"+str(pMut)+"_"+str(nPortfolio))

"""

In [ ]:
# Parameter
import matplotlib.pyplot as plt


listMaxGen = [50]
listNgen = [5,10,25,50,100]
listPMut = [0.2,0.4,0.5]
listPCo = [0.2,0.4,0.5]
listPortfolio = [50,100]
# Step 1 Generate Population
count = 0
for maxGen in listMaxGen :
    for nGen in listNgen :
        for pCo in listPCo :
            for pMut in listPMut :
                for nPortfolio in listPortfolio :
    
                    print(count+1)                    
                    print("Start_"+str(maxGen)+"_"+str(nGen)+"_"+str(pCo)+"_"+str(pMut)+"_"+str(nPortfolio))
                    print("End_"+str(maxGen)+"_"+str(nGen)+"_"+str(pCo)+"_"+str(pMut)+"_"+str(nPortfolio))
                    count+=1                 


In [ ]:
for maxGen in listMaxGen :
    for nGen in listNgen :
        for pCo in listPCo :
            for pMut in listPMut :
                for nPortfolio in listPortfolio :
                    print("crossover = ",pCo)
                    print("Mut =",pMut)
                    print("nGen = ",nGen)
                    print("nPortfolio = ",nPortfolio)
                    dfGraf = pd.read_csv("fungsi_"+str(nGen)+"_"+str(pCo)+"_"+str(pMut)+"_"+str(nPortfolio)+".csv")
                    x = dfGraf["x"].tolist()
                    y = dfGraf["y"].tolist()
                    plt.title("gen "+str(gen+1))
                    plt.xlabel('Diversity', fontsize=15)
                    plt.ylabel('Activtity', fontsize=15)
                    plt.scatter(y, x)
                    plt.show()

